<h1 style = "font-size: 30px; text-align: center;">AI Genetic Algorithm</h1>
<h2 style = "font-size: 25px; text-align: center;">Hospital Job Scheduling</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Name: Toktam Sameie</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Student Id: 810896054</h2>
<h4 style="text-align: center">Spring 1400</h4>

<h2 style = "font-size: 25px;">Notes</h2>

<h3>Tests: </h3>

<p style="text-indent :2em;"><b>Test1: </b><mark>Average time < 15s</mark></p>
<p style="text-indent :2em;"><b>Test2: </b><mark>Average time < 60s</mark></p>
    

<h3>Outputs: </h3>
<p style="text-indent :2em;">Respectively <mark>output1.txt</mark> and <mark>output2.txt</mark></p>

<br>

In this project, we are going to solve the schedule problem with the help of genetic algorithm.

<h2 style = "font-size: 25px;">Test Files</h2>

In [1]:
testFile1 = "test1.txt"
testFile2 = "test2.txt"

In [2]:
import random
import collections
import decimal
import copy
from IPython.display import HTML, display
import csv

<h2 style = "font-size: 25px;">Reading Test File Content</h2>

In this section, we extract the data from the input file and convert it to the required format

In [3]:
def readInput(testFile) :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    
    allShifts = []
    for i in range(2, days + 2):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        
        allShifts.append((morningReqs, eveningReqs, nightReqs))

    file.close()
    return [days, list(range(doctors)), maxCapacity, allShifts]

<h2 style = "font-size: 25px;">Job Scheduler</h2>

We use this function  to display the schedule in required format.

In [4]:
def output(listt):
    for i in range(len(listt)):
            print(*listt[i][0],sep=",",end=" ")
            print(*listt[i][1],sep=",",end=" ")
            print(*listt[i][2],sep=",")
           
  

In [5]:
def write_to_file(listt,name_of_file):
    with open(name_of_file, 'w') as f:
        for i in range(len(listt)):
                print(*listt[i][0],sep=",",end=" ", file=f)
                print(*listt[i][1],sep=",",end=" ", file=f)
                print(*listt[i][2],sep=",", file=f)

The JobScheduler class has some attribute based on which we design different methods.<br/>
In "generateInitialPopulation" method ,  initialize the "self.chromosomes" and then we use this attribute in other methods.<br/>
The "crossOver" method , take two chromosomes and based on a self.pc probability, performs night shift shifts between two chromosomes.<br/>
The "mutate" method , take one argument and based on a self.pm probability , changes the number of doctors in every shift based on the limit on the number of doctors per shift.<br/>
Then in "calculateFitness", we calculate the fitness of each chromosome based on the limits on shifts in this problem .<br/>
The "generateNewPopulation" method , generate new generation in this way, it first passes 16 chromosomes directly from the sorted chromosomes by fitness value to the next generation , and then randomly assigns the chromosomes to the crossOver function . in the next step, the generation generated in the previous step , is randomly given to the mutate function ... <br/>
In this way, a new generation is produced .<br/>
Then, in the "schedule" function, we continue to generate the generation until we reach the goal, which is the fitness = 1 .<br/>
At the end of each while loop, we sort the generated generation in reverse and check the value at the beginning of the list for the goal condition.


In [6]:
class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.allShifts = fileInfo[3]
        self.chromosomes = self.generateInitialPopulation()
        self.popSize = 300
        self.elitismPercentage = 16 
        self.pc = 0.15
        self.pm = 0.8  

    def generateInitialPopulation(self):
        chromosomes = []        
        counter =0 
        while counter < 300 :
            chromosome = []
            for i in range(self.days):  
                row = []
                for j in range(3):
                    numberOfDoctorsInEveryShift  = random.randint(self.allShifts[i][j][0],self.allShifts[i][j][1])
                    row.append(random.sample(range(1, self.doctors), numberOfDoctorsInEveryShift))
                chromosome.append(row)
                del row
            counter +=1
            fit = self.calculateFitness(chromosome)
            chromosomes.append((chromosome ,fit ))
            del chromosome
        return chromosomes
    
    def crossOver(self, chromosome1, chromosome2):
        new_chrom = [[0]*3]*self.days
        for i in range(self.days):
            new_chrom[i] = chromosome1[i]
            if float(decimal.Decimal(random.randrange(0, 1000))/1000) > self.pc :     
                new_chrom[i][2] = chromosome2[i][2]
            else :
                continue               
        return new_chrom
        
                
    def mutate(self, chromosome):
        new_chrom = copy.deepcopy(chromosome)
        for i in range(self.days):
            for j in range(3):
                if float(decimal.Decimal(random.randrange(0, 1000))/1000) > self.pm :
                    if len(chromosome[i][j]) < self.allShifts[i][j][1] :
                        rand = random.randint(0,self.doctors)
                        if rand not in chromosome[i][0] :
                            new_chrom[i][j].append(rand)
                            continue
                            
                    if len(chromosome[i][j]) > self.allShifts[i][j][0] :
                        rand = random.randint(0,len(chromosome[i][j])-1)
                        new_chrom[i][j].remove(chromosome[i][j][rand])
                        continue
                    else :
                        continue
                else:
                    continue
                    
        return new_chrom
        
        
    def calculateFitness(self, chromosome):
        conflicts = 0 
        all_shifts = []
        for i in range(self.days):            
            if i < self.days -1 :
                morn = list(set(chromosome[i][2]).intersection(chromosome[i+1][0]))
                noon = list(set(chromosome[i][2]).intersection(chromosome[i+1][1]))
                conflicts +=len(morn)               
                conflicts +=len(noon) 
            if i < self.days - 3 :
                night = list(set(chromosome[i][2]).intersection(chromosome[i+1][2] , chromosome[i+2][2]))
                conflicts +=len (night)  
            all_shifts +=  chromosome[i][0]+ chromosome[i][1] +chromosome[i][2]
        frequency = collections.Counter(all_shifts)
        val_list = list(frequency.values())
        for k in range(len(val_list)):
            if val_list[k] >9:
                  conflicts += val_list[k] - 9
        
        return 1/(conflicts +1)
    
    
    def generateNewPopulation(self,chromosomes):
        new_generation = []        
        new_generation = chromosomes[0:self.elitismPercentage]
        while len(new_generation) <= 300 :
            rand_element = random.sample(range(0, len(new_generation)-1), 2)
            elem1 = copy.deepcopy(chromosomes[rand_element[0]][0])
            elem2 = copy.deepcopy(chromosomes[rand_element[1]][0])
            chrom = self.crossOver(elem1,elem2) 
            fit = self.calculateFitness(chrom)
            FIT = copy.deepcopy(fit)
            if len(new_generation) < 300:
                new_generation.append((chrom , FIT))
            else :
                break
                 
        new_generation2 =[]  
        while len(new_generation2) <= 300:
            rand_elem = random.randint(0, len(new_generation)-1)
            elem = copy.deepcopy(new_generation[rand_elem][0])
            chrom = self.mutate(elem)
            fit = self.calculateFitness(chrom)
            FIT = copy.deepcopy(fit)
            new_generation2.append((chrom , FIT))
        self.chromosomes = new_generation2
        del new_generation
        answer = copy.deepcopy(new_generation2)
        del new_generation2
        return answer
    
    
    def schedule(self):
        start_chrom = copy.deepcopy(self.chromosomes)
        while True :
            new_generation = self.generateNewPopulation(start_chrom)
            del start_chrom
            new_generation.sort(key=lambda x:x[1], reverse=True)
            if new_generation[0][1] == 1:
                output(new_generation[0][0])
                return new_generation[0][0]
            else:
                start_chrom = copy.deepcopy(new_generation)
                del new_generation
                continue
        return


<h2 style = "font-size: 25px;">Execution</h2>

In [7]:
import time

fileInfo1 = readInput(testFile1)

start = time.time()

scheduler = JobScheduler(fileInfo1)
scheduler.generateInitialPopulation()
ans = scheduler.schedule()
write_to_file(ans,'output1.txt')
end = time.time()

print("test 1: ", '%.2f'%(end - start), 'sec')

6,5,1 1 4,6
3 3,2 4
3 1,6,0 5
6 4,3 5,4,2
6  2
3 0 1,6
4,5 2 3
0 4 4,6
5  
3 6 2,3,3
test 1:  1.69 sec


In [9]:
fileInfo2 = readInput(testFile2)

start = time.time()

scheduler = JobScheduler(fileInfo2)
scheduler.generateInitialPopulation()
ANS = scheduler.schedule()
write_to_file(ANS,'output2.txt')
end = time.time()

print("Test 2: ", '%.2f'%(end - start), 'sec')

6,4 5 8
0,2 7 9
5 8 8,6
 4,9,3,1 2,8
6 5,1 6
1,2 3,3 1,4
7 5,2 1,3
0 5,2,0,4,8 2,0,0
3 5,6,9,5 7,7
9,1 0 0,0,5
1,2 9,6,9 
2,4 7 
9,8 3,8,4 7,6
3 1,4 2,7,3
 6,8,8 7
Test 2:  15.04 sec
